In [2]:
from htmd import *

Please cite. HTMD: High-Throughput Molecular Dynamics for Molecular Discovery, J. Chem. Theory Comput., 2016, 12 (4), pp 1845-1852. 
http://pubs.acs.org/doi/abs/10.1021/acs.jctc.6b00049


Failed connecting to http://conda.anaconda.org. Cannot check for new HTMD versions.
You are on the latest HTMD version (1.0.26).


In [7]:
mol = Molecule('3PTB')

2016-06-14 11:36:55,081 - htmd.molecule.molecule - INFO - Attempting PDB query for 3PTB


KNOWN:
   - Donors:
        - ARG protonated* name NH1 NH2
        - LYS protonated* name NZ
   - Acceptors:
        - ASP deprotonated* name OD2 OD1
        - GLU deprotonated* name OE1 OE2

UNKNWON:
   - TYR name OH
   - HIS – There are 6 alternates for HIS, two rotomers and then 3 protonation states for each name ND1 NE2
   - CYS – disulfide bonded or not – name SG

CONFLICT:
   - ASN – There are 2 alternates – simple 180 degree rotations name ND2 OD1
   - GLN – There are 2 alternates – simple 180 degree rotations name OE1 NE2
   - SER name OG
   - THR name OG1

Backbone name O, N
   
UNCLASSIFIED: 
   - ALA
   - GLY
   - ILE
   - LEU
   - PRO 
   - VAL
   - MET

In [10]:
mol.view()

In [19]:
copied = mol.copy()

In [41]:
copied.filter('resname ARG LYS ASP GLU HIS CYS TYR')

In [44]:
rid = np.unique(copied.get('resid')) # ia.resid also works!
rn = copied.get('resname', "name CA")

In [53]:
for resname, resid in zip(rn, rid):
    if resname == "LYS":
        a = mol.get("resid", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG ND1 NH1 OD1 OD2 NZ NH2 and within 4 of name NZ and resid "+str(resid))
        print(a, "resid", resid)

[37 39 60] resid 60
[ 87 245] resid 87
[107 245 713] resid 107
[109] resid 109
[145 728] resid 145
[ 21 156] resid 156
[ 20 159] resid 159
[165 169] resid 169
[188] resid 188
[202 204] resid 204
[222] resid 222
[171 224] resid 224
[230] resid 230
[235 236 239] resid 239


In [48]:
np.unique(mol.get("name","resname LYS"))

array(['C', 'CA', 'CB', 'CD', 'CE', 'CG', 'N', 'NZ', 'O'], dtype=object)